# Cortex Agents for Multiple Tenants/Users
There are a number of use cases where we want to use the Cortex AI features of Snowflake but only give certain
users access to certain data in the database. 

For example, we may have sales data for our company, and we want
to create an application that lets our salespeople ask questions in a "talk-to-your-data" sort of fashion, but
we only want a salesperson to get access to the data about their customers and not other customers. Another
example is that we have an application that manages data for many tenants, and we want to allow a tenant to 
ask questions of their data, but not get any access to other tenants' data.

We can accomplish this using Snowflake, leveraging the Cortex Agents feature for AI and the Row Access Policy
(RAP) feature to restrict the data for the user.

This Notebook will walk you through the setup for a 3-tier web app that uses Cortex Agents to talk to your data
and manages sales data for multiple tenants. Users will log in to the application and only see results related
to the tenant they are associated with. The use case is managing restaurant sales data (how much each menu item
is selling at each restaurant within each chain). 

The high-level steps in the Notebook are:
* set up a warehouse, database, and schema for this example
* create a role for the example and grant it the necessary privileges to create the objects it needs
* create a set of menu items (using Cortex to generate descriptions)
* create a Cortex Search service for the menu items
* create daily sales totals for each store in each restaurant
* create a Programmatic Access Token (PAT) to allow our application to talk to Snowflake APIs
* create a Row Access Policy based on the value in a session variable and apply it to the items and orders tables
* print the environment variables that you will need to run the companion web app

## 1. Setup
First, we will create the `WAREHOUSE` that we can use for this project, as well as a 
`DATABASE` and `SCHEMA` to host the objects.


In [ ]:
USE ROLE accountadmin;
CREATE WAREHOUSE IF NOT EXISTS multisales_wh WITH WAREHOUSE_SIZE='XSMALL';
CREATE DATABASE IF NOT EXISTS multisales;
CREATE SCHEMA IF NOT EXISTS multisales.data;


Next, we create the `MULTISALES_RL` role that will own the objects and grant it the necessary permissions.

In [ ]:
USE ROLE accountadmin;
CREATE ROLE IF NOT EXISTS multisales_rl;
GRANT ROLE multisales_rl TO ROLE accountadmin;

GRANT USAGE ON WAREHOUSE multisales_wh TO ROLE multisales_rl;
GRANT ALL ON DATABASE multisales TO ROLE multisales_rl;
GRANT ALL ON SCHEMA multisales.data TO ROLE multisales_rl;
GRANT DATABASE ROLE snowflake.cortex_user TO ROLE multisales_rl;

## 2. Data
Next, we create the data for our example.

### Items
Now, we create the menu items for our restaurants. There are 3 restaurant chains:
* Alices Restaurant
* Bobs Place
* Charlies Diner

They sell a variety of sandwiches, burgers, pasta, and tacos. We will use Cortex to generate the descriptions of 
the menu items based on their item names.

In [ ]:
USE ROLE multisales_rl;
USE SCHEMA multisales.data;
CREATE OR REPLACE
TABLE items AS
    WITH items AS (
        SELECT 101::INT AS item_id, 'Alices Restaurant' AS tenant, 'Chicken Salad Sandwich' AS item
        UNION ALL
        SELECT 102::INT AS item_id, 'Alices Restaurant' AS tenant, 'Chicken Caesar Wrap' AS item
        UNION ALL
        SELECT 103::INT AS item_id, 'Alices Restaurant' AS tenant, 'Grilled Chicken Sandwich' AS item
        UNION ALL
        SELECT 104::INT AS item_id, 'Alices Restaurant' AS tenant, 'Cheeseburger' AS item
        UNION ALL
        SELECT 105::INT AS item_id, 'Alices Restaurant' AS tenant, 'Bacon Double Cheeseberger' AS item
        UNION ALL
        SELECT 106::INT AS item_id, 'Alices Restaurant' AS tenant, 'Swiss Bleu Burger' AS item
        UNION ALL
        SELECT 107::INT AS item_id, 'Alices Restaurant' AS tenant, 'Spaghetti Bolognese' AS item
        UNION ALL
        SELECT 108::INT AS item_id, 'Alices Restaurant' AS tenant, 'Pesto Totellini' AS item
        UNION ALL
    
        SELECT 201::INT AS item_id, 'Bobs Place' AS tenant, 'Fried Chicken Sandwich' AS item
        UNION ALL
        SELECT 202::INT AS item_id, 'Bobs Place' AS tenant, 'Chicken Bacon Ranch Sandwich' AS item
        UNION ALL
        SELECT 203::INT AS item_id, 'Bobs Place' AS tenant, 'Hamburger' AS item
        UNION ALL
        SELECT 204::INT AS item_id, 'Bobs Place' AS tenant, 'Cheddar Burger' AS item
        UNION ALL
        SELECT 205::INT AS item_id, 'Bobs Place' AS tenant, 'Swiss Burger' AS item
        UNION ALL
        SELECT 206::INT AS item_id, 'Bobs Place' AS tenant, 'Chicken Tacos' AS item
        UNION ALL
        SELECT 207::INT AS item_id, 'Bobs Place' AS tenant, 'Beef Tacos' AS item
        UNION ALL
        SELECT 208::INT AS item_id, 'Bobs Place' AS tenant, 'Veggie Tacos' AS item
        UNION ALL
    
        SELECT 301::INT AS item_id, 'Charlies Diner' AS tenant, 'Smoked Chicken Sandwich' AS item
        UNION ALL
        SELECT 302::INT AS item_id, 'Charlies Diner' AS tenant, 'Fried Korean Chicken Sandwich' AS item
        UNION ALL
        SELECT 303::INT AS item_id, 'Charlies Diner' AS tenant, 'Greek Chicken Wrap' AS item
        UNION ALL
        SELECT 304::INT AS item_id, 'Charlies Diner' AS tenant, 'Fried Fish Sandwich' AS item
        UNION ALL
        SELECT 305::INT AS item_id, 'Charlies Diner' AS tenant, 'Tuna Salad Sandwich' AS item
    )
    SELECT 
        item_id,
        tenant,
        item,
        AI_COMPLETE('claude-4-sonnet', 'You are writing menu descriptions for the following menu item: ' || item || '. The description should be no longer than 50 words. The description should entice customers to order the item')::VARCHAR AS item_description
    FROM items;

ALTER TABLE items SET CHANGE_TRACKING = TRUE;

### Orders
Next, we create the daily sales totals for each menu item at each store for each restaurant franchise.

In [ ]:
USE ROLE multisales_rl;
USE SCHEMA multisales.data;
CREATE OR REPLACE TABLE orders AS 
    WITH days AS (
        SELECT 
            DATEADD(day, seq4(), '2024-01-01'::DATE) AS date
        FROM TABLE (GENERATOR(ROWCOUNT => 365))
    )
    , tenant_popularity AS (
        SELECT 'Alices Restaurant' AS tenant, 500::INT AS popularity
        UNION ALL
        SELECT 'Bobs Place' AS tenant, 100::INT AS popularity
        UNION ALL
        SELECT 'Charlies Diner' AS tenant, 1000::INT AS popularity
    )
    , item_popularity AS (
        SELECT 
            i.item_id,
            i.item,
            tp.tenant,
            tp.popularity + NORMAL(0, 100, RANDOM(123)) AS popularity
        FROM items AS i
        JOIN tenant_popularity AS tp
          ON i.tenant = tp.tenant
    )
    , stores AS (
        SELECT 1001+seq4() AS store_id, 'Alices Restaurant' AS tenant
        FROM TABLE(GENERATOR(ROWCOUNT => 4))
        UNION ALL
        SELECT 2001+seq4() AS store_id, 'Bobs Place' AS tenant
        FROM TABLE(GENERATOR(ROWCOUNT => 3))
        UNION ALL
        SELECT 3001+seq4() AS store_id, 'Charlies Diner' AS tenant
        FROM TABLE(GENERATOR(ROWCOUNT => 5))
    )
    , orders AS (
        SELECT 
            d.date AS date,
            s.store_id,
            s.tenant,
            ip.item_id AS item_id,
            ip.item AS item,
            FLOOR(ip.popularity + NORMAL(0,100, RANDOM(234))) AS quantity
        FROM stores AS s
        JOIN item_popularity AS ip
          ON s.tenant = ip.tenant
        CROSS JOIN days AS d
    )
    SELECT
        * 
    FROM orders
;

### Cortex Search Services
We want to leveage Cortex Search to help Cortex Agents understand our questions and enable better
accuracy for answers. We will create two Cortex Search Services on the `ITEMS` table: one on the `ITEM` column and one 
on the `ITEM_DESCRIPTION` column.

In [ ]:
USE ROLE multisales_rl;
USE SCHEMA multisales.data;

CREATE OR REPLACE CORTEX SEARCH SERVICE items_search
    ON item
    ATTRIBUTES item_id, tenant, item_description
    WAREHOUSE = multisales_wh
    TARGET_LAG = '1 minute'
    AS (
        SELECT
            item_id,
            tenant,
            item,
            item_description
        FROM items
    )
;

CREATE OR REPLACE CORTEX SEARCH SERVICE item_description_search
    ON item_description
    ATTRIBUTES item_id, tenant, item
    WAREHOUSE = multisales_wh
    TARGET_LAG = '1 minute'
    AS (
        SELECT
            item_id,
            tenant,
            item,
            item_description
        FROM items
    )
;


We will need a `STAGE` to hold the Cortex Analyst semantic model for this project.

After you create the `STAGE`, go to the Snowsight UI and navigate to the `STAGE` in there to upload the
`MULTISALES.yaml` file that is included in the GitHub repo.

In [ ]:
USE ROLE multisales_rl;
USE SCHEMA multisales.data;

CREATE STAGE IF NOT EXISTS models DIRECTORY = (ENABLE = TRUE);

## 3. Row Access Policy
Now we can turn our attention to protecting the data so that a given tenant can only access their own data.

We will use an entitlement table to map which users (`Alice`, `Bob`, and `Charlie`) can access data for which
tenant (`Alices Restaurant`, `Bobs Place`, `Charlies Diner`). For now, each user can access their own 
restaurant's tenant. If, say, Alice hires a new employee, say Arlo, then we just need to add a row for 
Arlo allowing access to Alices Restaurant's data. It is quite simple and efficient to update the user access
by simply updating the data in the entitlement table.

Next, we will use a simple function to create an array of tenants that a given user has access to. This is 
essentially gathering all rows in the entitlement table for a given `USERID` and turning that into an
`ARRAY`. We will create this function as a _memoizable_ function so that it can efficiently be calculated
once and the result reused without having to recalculate every time.

Lastly, we create the actual Row Access Policy which will compare the value of the `REP` input to the
array of allowed tenants. We only use that logic if the current role is the `MULTISALES_RL`. This allows
us to access all of the data (assuming we have been granted `SELECT` permission on the table) if we access
it with a different role (e.g., `ACCOUNTADMIN`).

In [ ]:
USE ROLE multisales_rl;
USE SCHEMA multisales.data;

CREATE OR REPLACE TABLE orders_entitlement(userid VARCHAR, tenant VARCHAR);
INSERT INTO orders_entitlement(userid, tenant) VALUES
    ('Alice', 'Alices Restaurant'),
    ('Bob', 'Bobs Place'),
    ('Charlie', 'Charlies Diner')
;

CREATE OR REPLACE FUNCTION tenants(who STRING)
    RETURNS ARRAY
    MEMOIZABLE
    AS 'SELECT ARRAY_AGG(tenant) FROM orders_entitlement WHERE userid = who'
;

CREATE OR REPLACE ROW ACCESS POLICY rap_tenant
    AS (rep VARCHAR)
    RETURNS BOOLEAN ->
        CASE 
            WHEN 'MULTISALES_RL' = CURRENT_ROLE() THEN  ARRAY_CONTAINS(rep::VARIANT, tenants(GETVARIABLE('TENANT')))
            ELSE TRUE
        END
;
 

Now that we have created the Row Access Policy, we can apply that policy to both the `ITEMS` and `ORDERS` table. 
We will apply the policy to the `TENANT` column in both tables.

In [ ]:
USE ROLE multisales_rl;
USE SCHEMA multisales.data;

ALTER TABLE orders DROP ALL ROW ACCESS POLICIES; -- Just to make sure there aren't any others
ALTER TABLE orders ADD ROW ACCESS POLICY rap_tenant ON (tenant);

ALTER TABLE items DROP ALL ROW ACCESS POLICIES; -- Just to make sure there aren't any others
ALTER TABLE items ADD ROW ACCESS POLICY rap_tenant ON (tenant);

## 4. Programmatic Access Token
In order to enable our application to call Snowflake APIs (both the Cortex Agent API and the SQL API) we will
use a Programmatic Access Token (PAT). We can restrict this PAT to using a particular role, so we will set that
restriction to the `MULTISALES_RL` role.

In [ ]:
ALTER USER ADD PROGRAMMATIC ACCESS TOKEN multisales_pat ROLE_RESTRICTION = 'MULTISALES_RL' DAYS_TO_EXPIRY = 365;

## 5. Companion App
There is a companion 3-tier web app with this example. It has a frontend and a backend, both implemented
using Node.js. You can follow the README to see how to set it up. You will need to set some environment
variables in the terminal when you start both the frontend and the backend. 

Run the following cell to get the environment variables to use. Copy the output for the backend to the 
`/backend/.env` file.

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.connector import DictCursor

# Get the PAT from the `PAT` cell
pat = PAT.to_pandas().iloc[0].to_dict()['token_secret']

# Get the account locator via SQL
session = get_active_session()
cur = session.connection.cursor(DictCursor)
d = cur.execute("SELECT current_account() AS account").fetchall()
account = d[0]["ACCOUNT"]

st.markdown(f"""
### Backend
Create (if needed) the file `/backend/.env` and copy the following contents to it:
```
# Backend Server Configuration
BACKEND_PORT=4000

# Snowflake Connection
SNOWFLAKE_URL="https://{account}.snowflakecomputing.com"
SNOWFLAKE_PAT="{pat}"
SNOWFLAKE_WAREHOUSE="MULTISALES_WH"

# Cortex Agent Resources
SEMANTIC_MODEL_PATH="@MULTISALES.DATA.MODELS/MULTISALES.yaml"
SEARCH_SERVICE_PATH="MULTISALES.DATA.ITEM_DESCRIPTION_SEARCH
VICE_PATH="MULTISALES.DATA.ITEM_DESCRIPTION_SEARCH
```
""")
